In [ ]:
!pip install split-folders

In [ ]:
import os
import shutil
import itertools
import io
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

import tensorflow as tf
import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import (
    EfficientNetB0, MobileNetV2, ResNet50, InceptionV3
)
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_pre
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mob_pre
from tensorflow.keras.applications.resnet50 import preprocess_input as res_pre
from tensorflow.keras.applications.inception_v3 import preprocess_input as inc_pre

import splitfolders

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
import zipfile, os

uploaded = files.upload()
zip_name = list(uploaded.keys())[0]

with zipfile.ZipFile(zip_name, 'r') as z:
    z.extractall('/content/data')

!find /content/data -maxdepth 3 -type d -print

In [ ]:

DATA_DIR = '/content/data' #@param {type:"string"}
USE_LABELS_CSV = False #@param {type:"boolean"}
LABELS_CSV_PATH = '/content/data/my_dataset/labels.csv' #@param {type:"string"}

BACKBONE = 'MobileNetV2' #@param ["EfficientNetB0", "MobileNetV2", "ResNet50", "InceptionV3"]

# Train parameters
IMG_SIZE = 224 #@param {type:"integer"}
BATCH_SIZE = 32 #@param {type:"integer"}
EPOCHS = 15 #@param {type:"integer"}
VAL_SPLIT = 0.2 #@param {type:"number"}
AUTO_SPLIT_IF_NO_VAL = True #@param {type:"boolean"}
AUGMENTATION = False #@param {type:"boolean"}
LEARNING_RATE = 0.0005 #@param {type:"number"}

OUTPUT_DIR = '/content/output' #@param {type:"string"}
os.makedirs(OUTPUT_DIR, exist_ok=True)
print('Ayarlar yüklendi.')

In [ ]:
train_dir = os.path.join(DATA_DIR, 'train')
val_dir = os.path.join(DATA_DIR, 'val')

if os.path.isdir(train_dir) and (not os.path.isdir(val_dir)) and AUTO_SPLIT_IF_NO_VAL and not USE_LABELS_CSV:
    tmp_out = '/content/data_split'
    if os.path.exists(tmp_out):
        shutil.rmtree(tmp_out)
    splitfolders.ratio(train_dir, output=tmp_out, seed=42, ratio=(1-VAL_SPLIT, VAL_SPLIT))
    DATA_DIR = tmp_out
    train_dir = os.path.join(DATA_DIR, 'train')
    val_dir = os.path.join(DATA_DIR, 'val')

print('train_dir:', train_dir)
print('val_dir  :', val_dir)

IMG_SIZE = int(IMG_SIZE)
img_size = (IMG_SIZE, IMG_SIZE)

if not USE_LABELS_CSV:
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_dir, image_size=img_size, batch_size=BATCH_SIZE, shuffle=True)
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        val_dir, image_size=img_size, batch_size=BATCH_SIZE, shuffle=False)
    class_names = train_ds.class_names

    with open("class_names.txt", "w") as f:
        for cls in class_names:
            f.write(cls + "\n")
    print("✅ class_names.txt created, content:", class_names)
else:

    df = pd.read_csv(LABELS_CSV_PATH)
    class_names = sorted(df['label'].unique().tolist())
    class_to_idx = {c:i for i,c in enumerate(class_names)}

    paths = df['filepath'].tolist()
    labels = df['label'].map(class_to_idx).values

    def load_image(path, label):
        raw = tf.io.read_file(os.path.join(DATA_DIR, path))
        img = tf.io.decode_image(raw, channels=3)
        img = tf.image.resize(img, img_size)
        img.set_shape((IMG_SIZE, IMG_SIZE, 3))
        return img, label

    full_ds = tf.data.Dataset.from_tensor_slices((paths, labels)).map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    n_total = len(paths)
    n_val = int(n_total * VAL_SPLIT)
    val_ds = full_ds.take(n_val).batch(BATCH_SIZE)
    train_ds = full_ds.skip(n_val).shuffle(1000).batch(BATCH_SIZE)


train_ds = train_ds.cache().prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(tf.data.AUTOTUNE)

print('Sınıflar:', class_names)
NUM_CLASSES = len(class_names)

In [ ]:
btn = widgets.Button(
    description="Download class_names.txt",
    button_style="success",
    icon="download",
    layout=widgets.Layout(width="auto")
)

def _on_click(b):
    files.download("class_names.txt")

btn.on_click(_on_click)
display(btn)

In [ ]:
ds_tmp = tf.keras.utils.image_dataset_from_directory(
    train_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=32, shuffle=False
)
correct_names = ds_tmp.class_names  # alfabetik klasör sırası
print("Correct class order:", correct_names)

with open("class_names.txt", "w") as f:
    for cls in correct_names:
        f.write(cls + "\n")
print("✅ Rebuilt class_names.txt")

# (İstersen indir)
from google.colab import files
files.download("class_names.txt")

In [ ]:

preprocess_map = {
    'EfficientNetB0': eff_pre,
    'MobileNetV2': mob_pre,
    'ResNet50': res_pre,
    'InceptionV3': inc_pre
}

base_map = {
    'EfficientNetB0': EfficientNetB0,
    'MobileNetV2': MobileNetV2,
    'ResNet50': ResNet50,
    'InceptionV3': InceptionV3
}

pre_fn = preprocess_map[BACKBONE]
BaseClass = base_map[BACKBONE]


if AUGMENTATION:
    aug = tf.keras.Sequential([
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.1),
    ], name="aug")
else:

    aug = layers.Lambda(lambda x: x, name="identity_aug")


pre = tf.keras.layers.Lambda(pre_fn)

base = BaseClass(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet')
base.trainable = False
inp = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = aug(inp)
x = pre(x)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
out = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = tf.keras.Model(inp, out)
model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ckpt_path = os.path.join(OUTPUT_DIR, f'{BACKBONE}_best.keras')
callbacks = [
    ModelCheckpoint(ckpt_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
]

hist = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)

base.trainable = True
for layer in base.layers[:-20]:
    layer.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE/10),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

hist_ft = model.fit(train_ds, validation_data=val_ds, epochs=max(3, EPOCHS//3), callbacks=callbacks)

print('EBest model:', ckpt_path)

In [ ]:
def combine_history(h1, h2):
    history = {}
    for key in h1.history.keys():
        history[key] = h1.history[key] + h2.history[key]
    return history

history = combine_history(hist, hist_ft)

# Plot training vs validation accuracy and loss
plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()


In [ ]:

y_true = []
y_pred = []
for imgs, labels in val_ds:
    preds = model.predict(imgs, verbose=0)
    y_true.extend(labels.numpy().tolist())
    y_pred.extend(np.argmax(preds, axis=1).tolist())

cm = confusion_matrix(y_true, y_pred)
print(classification_report(y_true, y_pred, target_names=class_names))

plt.figure(figsize=(6, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(NUM_CLASSES)
plt.xticks(tick_marks, class_names, rotation=45, ha='right')
plt.yticks(tick_marks, class_names)


thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
import tensorflow as tf, os

# Keras
keras_path = os.path.join(OUTPUT_DIR, f'{BACKBONE}_final.keras')
model.save(keras_path)

# SavedModel
sm_path = os.path.join(OUTPUT_DIR, f'{BACKBONE}_savedmodel')
model.export(sm_path)

# TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(sm_path)
tflite_model = converter.convert()
with open(os.path.join(OUTPUT_DIR, f'{BACKBONE}.tflite'), 'wb') as f:
    f.write(tflite_model)

print('Kayıt klasörü:', OUTPUT_DIR)

In [ ]:
def _zip_dir(src_dir, zip_path):
    # Remove existing zip if present
    if os.path.exists(zip_path):
        os.remove(zip_path)
    base, _ = os.path.splitext(zip_path)
    # shutil.make_archive wants base name without .zip + format
    shutil.make_archive(base, 'zip', root_dir=src_dir)
    return zip_path

def _safe_download(path):
    if os.path.exists(path):
        files.download(path)
        print(f"Downloading: {os.path.basename(path)}")
    else:
        print(f"Not found: {path}")

import ipywidgets as widgets

btn_keras = widgets.Button(
    description="Download .keras",
    button_style='primary',
    layout=widgets.Layout(width="500px", height="100px")
)

btn_tflite = widgets.Button(
    description="Download .tflite",
    button_style='primary',
    layout=widgets.Layout(width="500px", height="100px")
)

btn_savedmodel = widgets.Button(
    description="Download SavedModel (.zip)",
    button_style='info',
    layout=widgets.Layout(width="500px", height="100px")
)

btn_all = widgets.Button(
    description="Download ALL outputs (.zip)",
    button_style='success',
    layout=widgets.Layout(width="500px", height="60px")
)

widgets.HBox([btn_keras, btn_tflite, btn_savedmodel, btn_all])


# --- Paths (re-derive defensively in case the user runs this cell alone) ---
keras_path = os.path.join(OUTPUT_DIR, f"{BACKBONE}_final.keras")
tflite_path = os.path.join(OUTPUT_DIR, f"{BACKBONE}.tflite")
sm_dir     = os.path.join(OUTPUT_DIR, f"{BACKBONE}_savedmodel")

savedmodel_zip = os.path.join(OUTPUT_DIR, f"{BACKBONE}_savedmodel.zip")
all_outputs_zip = os.path.join(OUTPUT_DIR, f"{BACKBONE}_all_outputs.zip")

# --- Buttons ---
btn_keras = widgets.Button(description="Download .keras", button_style='primary')
btn_tflite = widgets.Button(description="Download .tflite", button_style='primary')
btn_savedmodel = widgets.Button(description="Download SavedModel (.zip)", button_style='primary')
btn_all = widgets.Button(description="Download ALL outputs (.zip)", button_style='primary')

out = widgets.Output()

# --- Callbacks ---
def on_click_keras(b):
    with out:
        _safe_download(keras_path)

def on_click_tflite(b):
    with out:
        _safe_download(tflite_path)

def on_click_savedmodel(b):
    with out:
        if os.path.isdir(sm_dir):
            print("Zipping SavedModel directory...")
            _zip_dir(sm_dir, savedmodel_zip)
            _safe_download(savedmodel_zip)
        else:
            print(f"SavedModel directory not found: {sm_dir}")

def on_click_all(b):
    with out:
        if os.path.isdir(OUTPUT_DIR):
            print("Zipping all outputs...")
            _zip_dir(OUTPUT_DIR, all_outputs_zip)
            _safe_download(all_outputs_zip)
        else:
            print(f"Output folder not found: {OUTPUT_DIR}")

btn_keras.on_click(on_click_keras)
btn_tflite.on_click(on_click_tflite)
btn_savedmodel.on_click(on_click_savedmodel)
btn_all.on_click(on_click_all)

# --- UI layout ---
box = widgets.VBox([
    widgets.HTML("<h4>Download trained model files</h4>"
                 "<p>Click a button to download. If a file or folder is missing, run the training cells first.</p>"),
    widgets.HBox([btn_keras, btn_tflite]),
    widgets.HBox([btn_savedmodel, btn_all]),
    out
])

display(box)
